# Pathfinder Phase Workshop: Finding Data

__Description & purpose__: This Notebook is designed to showcase the functionality of the Earth Observation Data Hub (EODH) as the project approaches the end of the Pathfinder Phase. It provides a snapshot of the Hub, the `pyeodh` API client and the various datasets as of February 2025.   

__Author(s)__: Alastair Graham, Dusan Figala

__Date created__: 2025-02-18

__Date last modified__: 2025-02-20

__Licence__: This notebook is licensed under [Creative Commons Attribution-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-sa/4.0/).  The code is released using the [BSD-2-Clause](https://www.tldrlegal.com/license/bsd-2-clause-license-freebsd) license.


<span style="font-size:0.75em;">
Copyright (c) , All rights reserved.</span>

<span style="font-size:0.75em;">
Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:</span>

<span style="font-size:0.75em;">
Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer. Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution. THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE.</span>

# Visual data discovery

# STAC Browser

The first thing to do is find some data. Initially we will be using the current Catalogue User Interface which is an implementation of the STAC Browser (**Note**: a replacement user interface is in development, as demonstrated in the Workshop). When visiting the Catalogue you should see a page not too dissimilar to the following screenshot.

![](images/07a.png "Data Catalogue")

Click on the `Search` button in the top left of the page (next to *Browse*) and start to enter some details under `search for items`. We will look for data between 12 Nov and 19 Nov 2023, in the Thetford area, in the sentinel2_ard collection. The details are shown in the next screenshot. 

![](images/08a.png "Data search")

Once you click `Submit` the the search should return a series of image items to the right of the page, as shown below. 

![](images/09a.png "Search result")


We are looking for the following item (S2A_20231117_latn527lone0008_T30UYD_ORB137_20231117131218_utm30n_osgb) and the reference will be as shown here:

![](images/10a.png "Item information")

Click on the relevant item to find the assets within it. There are a number of assets (data layers, metadata, thumbnail etc.) within the item. Take some time to investigate what exists. The two we are interested in here are `thumbnail` and `cog` (the cog holds the image data). The image below shows how to copy the URL to the COG data: either using the button on the left or copying the path in the text box on the right.

Check that you have found the datset we are interested in:

* Thumbnail: https://dap.ceda.ac.uk/neodc/sentinel_ard/data/sentinel_2/2023/11/17/S2A_20231117_latn527lone0008_T30UYD_ORB137_20231117131218_utm30n_osgb_vmsk_sharp_rad_srefdem_stdsref_thumbnail.jpg (you can open this in a web browser and it should look like the image below)
* Dataset: https://dap.ceda.ac.uk/neodc/sentinel_ard/data/sentinel_2/2023/11/17/S2A_20231117_latn527lone0008_T30UYD_ORB137_20231117131218_utm30n_osgb_vmsk_sharp_rad_srefdem_stdsref.tif

![](images/10.png "Item thumbnail")

Take some time to click around the listed datasets to see what is included and accessible. 

**Note** that **not all collections contain accessible items**.

# Coded data discovery

There are a number of API endpoints that are exposed by the EODH. Oxidian have developed a Python API Client, `pyeodh`, that makes the Hub's API endpoints available to Python users. `pyeodh` is available on PyPi (https://pypi.org/project/pyeodh/) and can be installed using `pip`. Documentation for the API Client is available at: https://pyeodh.readthedocs.io/en/latest/api.html

We will use `pyeodh` throughout this workshop.

## Presentation set up

The following cell only needs to be run on the EODH AppHub.  If you have a local Python environment running, please install the required packages as you would normally e.g. using `mamba`, `poetry` etc.

In [ ]:
# If needed you can install a package in the current AppHub Jupyter environment using pip
# For instance, we will need at least the following libraries

import sys
!{sys.executable} -m pip install --upgrade pyeodh geopandas shapely matplotlib numpy pillow folium

In [ ]:
# Imports
import pyeodh

import shapely as sh 
import geopandas as gpd
import folium

import urllib.request
from requests.exceptions import HTTPError

from PIL import Image
from io import BytesIO

Having imported the necessary libraries the next task is to set up the locations of the areas of interest.
Having created the AOI points the user needs to connect to the Resource Catalogue so that they can start to find some data.

In [ ]:
# Areas of Interest
thet_pnt = sh.Point(0.6715892933273722, 52.414471075812315) # a site near Thetford

In [ ]:
# Optional cell
# If you want to see these points on a map run this cell
# You may need to run the notebook through a service such as nbviewer: https://nbviewer.org/

# Create a map (m) centered on the point
center_lat = (thet_pnt.y)
center_lon = (thet_pnt.x)

m = folium.Map(location=[center_lat, center_lon], zoom_start=10)

# Add markers for the point
folium.Marker([thet_pnt.y, thet_pnt.x], popup="Thetford Site", icon=folium.Icon(color="green")).add_to(m)

# Step 4: Display the map
m

In [ ]:
# Connect to the Hub
# base_url can be changed to optionally specify a different server, such as test.eodatahub

client = pyeodh.Client(
    base_url="https://staging.eodatahub.org.uk"
).get_catalog_service()

In [ ]:
# Print a list of the collections held in the Resource Catalogue (their id and description).
# As the Resource Catalogue fills and development continues, the number of collections and the richness of their descriptions will increase

for index, collect in enumerate(client.get_collections(), start=1):
    print(f"{index} -- {collect.id}: {collect.description}")

The dataset that we are interested in for the purposes of this workshop is `sentinel2_ard`. As seen from the output from the previous cell, we can see that the description of the dataset is as follows:

_These data have been created by the Department for Environment, Food and Rural Affairs (Defra) and Joint Nature Conservation Committee (JNCC) in order to cost-effectively provide high quality, Analysis Ready Data (ARD) for a wide range of applications. The dataset contains modified Copernicus Sentinel-2 (Level 1C data processed into a surface reflectance product using ARCSI software (Level 2))._

In [ ]:
# The next thing to do is find some open data
# For this workshop we want to find Sentinel-2 analysis ready (ARD) imagery near Thetford

# First we just want to understand the timespan of the dataset which is reported from the STAC collection record
sentinel2_ard = client.get_catalog("supported-datasets/catalogs/ceda-stac-catalogue").get_collection('sentinel2_ard')
sentinel2_ard.get_items()

print('DATASET TEMPORAL EXTENT: ', [str(d) for d in sentinel2_ard.extent.temporal.intervals[0]])

In [ ]:
# Now we want to access the first few items and see what they are called, when the image was collected and how much cloud there is  

lim = 10

for i, item in enumerate(sentinel2_ard.get_items()):
    if i >= lim:
        break
    print(item.id, item.properties['datetime'], item.properties['eo:cloud_cover'])

The previous cell shows us that we are able to access Sentinel 2 ARD data and find out a number of bits of information about the item. If you are interested in seeing what other information is accessible, have a look at the 

* collection endpoint: https://staging.eodatahub.org.uk/api/catalogue/stac/catalogs/supported-datasets/catalogs/ceda-stac-catalogue/collections/sentinel2_ard
* items endpoint: https://staging.eodatahub.org.uk/api/catalogue/stac/catalogs/supported-datasets/catalogs/ceda-stac-catalogue/collections/sentinel2_ard/items

In [ ]:
# To find specific imagery for the Thetford site we need to add the intersects parameter. We set this to be our AOI point.

items = client.search(
    collections=['sentinel2_ard'],
    catalog_paths=["supported-datasets/catalogs/ceda-stac-catalogue"],
    intersects=thet_pnt,
    query=[
        'start_datetime>=2023-11-01',
        'end_datetime<=2023-11-30', 
    ],
)

# We can then count the number of items returned by the search 
#print('Number of items found: ', items.total_count)

total_items = sum(1 for _ in items)
print(f"Total items: {total_items}")


In [ ]:
# We can print out the item names so that we understand which images we are looking at

for item in items:
    print(f"Item ID: {item.id}")


Once we have found the intersecting images and know their names we can choose the image item that we are interested. For the purposes of this exercise this is T31UCU. Now we need to know what assets are held for that item. The following code prints out all the STAC information linked to that item.

In [ ]:
for item in items[:1]:  # Process only the first item
    print(f"Item ID: {item.id}")
    print("Assets:")
    
    if not item.assets:
        print("  No assets available.")
    else:
        for asset_key, asset in item.assets.items():
            print(f"  - {asset_key}: {asset.to_dict()}")  # Convert asset to dict for readable output
            print("-" * 40)  # Separator for better readability

From this we can see that there is a thumbnail. Before we do anything with the full image data or associated assets it would be good to understand the image. We can extract the URL to the thumbnail and view the attached image file.

In [ ]:
tn_url = None

for item in items[:1]:  # Process only the first item
    #print(f"Item ID: {item.id}")
    #print("Assets:")
    
    if not item.assets:
        print("  No assets available.")
    else:
        for asset_key, asset in item.assets.items():
            #print(f"  - {asset_key}: {asset.to_dict()}")  # Convert asset to dict for readable output
            if asset_key == "thumbnail":
                tn_url = asset.href  # Directly access the href attribute
    
    #print("-" * 40)  # Separator for better readability
print(tn_url)

In [ ]:
# We can use this information to view the image thumbnail

# Here we open the remote URL, read the data and dislay the thumbnail 
with urllib.request.urlopen(tn_url) as url:
    img = Image.open(BytesIO(url.read()))

display(img)

This shows that we can relatively easily interrogate the Resource Catalogue and filter the results so that we can find the data we require in the EODH. With a bit of tweaking of the code the user could also generate a list of assets and accompanying URLs to the datasets (for this and other datasets). 

Now we want to see what commercial data exists. 

In [ ]:
# Find some Airbus commercial data: SPOT

count = 0
lim = 15

for i in client.search(
    catalog_paths=["supported-datasets/catalogs/airbus"],
    collections=["airbus_spot_data"],
):
    if count >= lim:
        break
    print(i.id)
    count += 1


In [ ]:
# Find some Planet commercial data: Planetscope Scene
count = 0
try:
    cat = client.get_catalog("supported-datasets/catalogs/planet")
    for i in cat.search(collections=["PSScene"]):
        if count >= lim:
            break
        print(i.id)
        count+= 1
except HTTPError as e:
    if "422 Client Error: Unprocessable Entity" in str(e):
        print("Skipping E422")
    else:
        raise  # Re-raise other errors

The final step would be to use the ordering service integrated into the EODH resource catalogue to purchase the required commercial imagery. This would be stored in a users workspace and could then be used in specific workflows or for data analytics (depending on licence restrictions).

For the purposes of this workshop we looked at the different commercial datasets offline in QGIS. 